In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
import PIL
import io
from keras.applications import resnet50
import h5py
import matplotlib.image as mat_img
from PIL import Image
import urllib.request
from keras.preprocessing.image import load_img, img_to_array
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
train_df=pd.read_excel('Training_Data.xlsx')
train_df.head()

In [ ]:
#Save training images
#def save_images(url):
#    import re
#    import os
#    with urllib.request.urlopen(url) as URL:
#        with open('temp.jpg','wb') as f:
#            f.write(URL.read())
#        img=Image.open('temp.jpg').convert('RGB')
#        img=img.resize((224,224))
#        category=train_df[train_df['ImageUrl']==url]['CategoryName'].tolist()[0]
#        if os.path.isdir('train_images/'+category) == False:
#            os.makedirs('train_images/'+category)
#        img.save('train_images/'+category+'/'+url.split('/')[-1])
#    return train_df[train_df['ImageUrl']==url]['Title'].tolist()[0]
#title = train_df['ImageUrl'].map(save_images)

In [ ]:
import pathlib
data_dir = pathlib.Path('train_images')
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

In [ ]:
def get_name(path):
    return path.split('/')[-1]

In [ ]:
train_df['Name']=train_df['ImageUrl'].map(get_name)

In [ ]:
train_df.head()

In [ ]:
from tensorflow.python.data.ops import dataset_ops
from tensorflow.python.keras.layers.preprocessing import image_preprocessing
from tensorflow.python.keras.preprocessing import dataset_utils
from tensorflow.python.ops import image_ops
from tensorflow.python.ops import io_ops
from tensorflow.python.util.tf_export import keras_export
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy


ALLOWLIST_FORMATS = ('.bmp', '.gif', '.jpeg', '.jpg', '.png')

def image_dataset_from_directory(directory,dataframe,
                                 labels='inferred',
                                 label_mode='int',
                                 class_names=None,
                                 color_mode='rgb',
                                 batch_size=32,
                                 image_size=(256, 256),
                                 shuffle=True,
                                 seed=None,
                                 validation_split=None,
                                 subset=None,
                                 interpolation='bilinear',
                                 follow_links=False):

    if labels != 'inferred':
        if not isinstance(labels, (list, tuple)):
            raise ValueError(
                  '`labels` argument should be a list/tuple of integer labels, of '
                  'the same size as the number of image files in the target '
                  'directory. If you wish to infer the labels from the subdirectory '
                  'names in the target directory, pass `labels="inferred"`. '
                  'If you wish to get a dataset that only contains images '
                  '(no labels), pass `label_mode=None`.')
        if class_names:
              raise ValueError('You can only pass `class_names` if the labels are '
                               'inferred from the subdirectory names in the target '
                               'directory (`labels="inferred"`).')
    if label_mode not in {'int', 'categorical', 'binary', None}:
        raise ValueError(
            '`label_mode` argument must be one of "int", "categorical", "binary", '
            'or None. Received: %s' % (label_mode,))
    if color_mode == 'rgb':
        num_channels = 3
    elif color_mode == 'rgba':
        num_channels = 4
    elif color_mode == 'grayscale':
        num_channels = 1
    else:
        raise ValueError(
            '`color_mode` must be one of {"rbg", "rgba", "grayscale"}. '
            'Received: %s' % (color_mode,))
    interpolation = image_preprocessing.get_interpolation(interpolation)
    dataset_utils.check_validation_split_arg(
        validation_split, subset, shuffle, seed)

    if seed is None:
        seed = np.random.randint(1e6)
    image_paths, labels, class_names = dataset_utils.index_directory(
        directory,
        labels,
        formats=ALLOWLIST_FORMATS,
        class_names=class_names,
        shuffle=shuffle,
        seed=seed,
        follow_links=follow_links)

    if label_mode == 'binary' and len(class_names) != 2:
        raise ValueError(
            'When passing `label_mode="binary", there must exactly 2 classes. '
            'Found the following classes: %s' % (class_names,))

    image_paths, labels = dataset_utils.get_training_or_validation_split(
        image_paths, labels, validation_split, subset)
    
    img_ds,text_ds,label_ds = paths_and_labels_to_dataset(
        image_paths=image_paths,
        image_size=image_size,
        num_channels=num_channels,
        labels=labels,
        label_mode=label_mode,
        num_classes=len(class_names),
        interpolation=interpolation,
        df=dataframe)
    #if shuffle:
    #    # Shuffle locally at each iteration
    #    img_ds = img_ds.shuffle(buffer_size=batch_size * 8, seed=seed)
    #    text_ds = text_ds.shuffle(buffer_size=batch_size * 8, seed=seed)
    #    label_ds = label_ds.shuffle(buffer_size=batch_size * 8, seed=seed)
    #img_ds = img_ds.batch(batch_size)
    #text_ds = text_ds.batch(batch_size)
    #label_ds = label_ds.batch(batch_size)
    # Users may need to reference `class_names`.
    #label_ds.class_names = class_names
    # Include file paths for images as attribute.
    #img_ds.file_paths = image_paths
    return img_ds,text_ds,label_ds


def paths_and_labels_to_dataset(image_paths,
                                image_size,
                                num_channels,
                                labels,
                                label_mode,
                                num_classes,
                                interpolation,
                                df):
    # TODO(fchollet): consider making num_parallel_calls settable
    path_ds = dataset_ops.Dataset.from_tensor_slices(image_paths)
    
    text_vectorizer = tf.keras.layers.experimental.preprocessing.TextVectorization(output_mode="tf-idf")
    text_vectorizer.adapt(df['Title'].tolist())
    
    text_list = [get_text(image_paths[i],df) for i in range(len(image_paths))]
    text_ds = np.array([sum(text_vectorizer(i)).numpy() for i in text_list])#dataset_ops.Dataset.from_tensor_slices(text_list)
    img_ds = np.array([path_to_image(image_paths[i],image_size,num_channels,interpolation).numpy() for i in range(len(image_paths))])#path_ds.map(lambda x: path_to_image(x, image_size, num_channels, interpolation))
    if label_mode:
        label_ds = np.array([tf.one_hot(i,63).numpy() for i in labels])#dataset_utils.labels_to_dataset(labels, label_mode, num_classes)
        #img_txt_ds = dataset_ops.Dataset.zip((img_ds,text_ds))
        #img_txt_ds = dataset_ops.Dataset.zip((img_txt_ds,label_ds))
    return img_ds,text_ds,label_ds


def path_to_image(path, image_size, num_channels, interpolation):
    img = io_ops.read_file(path)
    img = image_ops.decode_image(
        img, channels=num_channels, expand_animations=False)
    img = image_ops.resize_images_v2(img, image_size, method=interpolation)
    img.set_shape((image_size[0], image_size[1], num_channels))
    return img

def get_text(path,df):
    path = pathlib.Path(path)
    name = path.parts[-1]
    text = df[df['Name']==name]['Title'].to_string(index=False)
    return text

In [ ]:
batch_size = 32
img_height = 224
img_width = 224

In [ ]:
train_img_ds,train_text_ds,train_label_ds = image_dataset_from_directory(
            data_dir,dataframe = train_df,
            validation_split=0.2,
            subset = "training",
            seed = 123,
            image_size = (img_height,img_width),
            batch_size = batch_size)

In [ ]:
val_img_ds,val_text_ds,val_label_ds = image_dataset_from_directory(
            data_dir,dataframe = train_df,
            validation_split=0.2,
            subset = "validation",
            seed = 123,
            image_size = (img_height,img_width),
            batch_size = batch_size)

In [ ]:
preprocess_input = tf.keras.applications.resnet_v2.preprocess_input
image_size = (img_height,img_width)
image_shape = image_size+(3,)
base_model = tf.keras.applications.ResNet50V2(input_shape=image_shape,
                                              include_top=False,weights='imagenet')
base_model.trainable = False
base_model.summary()

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

In [ ]:
img_inputs = tf.keras.Input(shape=(224, 224, 3))
img = preprocess_input(img_inputs)
img = base_model(img, training=False)
img = global_average_layer(img)
img = tf.keras.Model(inputs=img_inputs,outputs=img)

txt_inputs = tf.keras.Input(shape=(13512))
txt = tf.keras.layers.Dense(1)(txt_inputs)
txt = tf.keras.Model(inputs=txt_inputs,outputs=txt)

x = tf.keras.layers.concatenate([img.output,txt.output])

x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(63)(x)
model = tf.keras.Model(inputs = [img.input,txt.input], outputs=outputs)

In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(x=[train_img_ds, train_text_ds], y=train_label_ds,
          validation_data=([val_img_ds, val_text_ds], val_label_ds),epochs=100)